In [ ]:
import requests
import json
import psycopg2
import time

def get_transaction_messages(bheight):
    transactions = []
    url = f"https://rest.comdex.one/cosmos/tx/v1beta1/txs?events=tx.height={bheight}"
    txs_res_arr = requests.get(url).json()["tx_responses"]
    print(len(txs_res_arr))
    return (txs_res_arr)

#Connect to postgre 
Dir = psycopg2.connect("dbname=Comdex user=postgres password=Ambuj@123 host=localhost port=5432")

                
def get_txs_request(bheight):

    
    txs_res_arr = get_transaction_messages(bheight)

    for i in range(len(txs_res_arr)):
        print("printing_>", i)
        txs_hash = txs_res_arr[i]["txhash"]
        msg_typ = txs_res_arr[i]["tx"]["body"]["messages"][0]["@type"]
        height = txs_res_arr[i]["height"]
        
        cur = Dir.cursor()
        
        #entry to main table always goes irrespective of the transactions
        cur.execute("INSERT INTO transactions (txs_hash, msg_typ, height) VALUES (%s, %s, %s)",
                        (txs_hash, msg_typ, height))

        #To push message type - transfer
        if msg_typ == "/ibc.applications.transfer.v1.MsgTransfer":
            sender = txs_res_arr[i]["tx"]["body"]["messages"][0]["sender"]
            receiver = txs_res_arr[i]["tx"]["body"]["messages"][0]["receiver"]
            amount = txs_res_arr[i]["tx"]["body"]["messages"][0]["token"]['amount']
            token_denom = txs_res_arr[i]["tx"]["body"]["messages"][0]["token"]['denom']
            
            cur.execute("INSERT INTO transfer (txs_hash, sender, receiver, amount, token_denom, height) VALUES (%s, %s, %s, %s, %s, %s)",
                        (txs_hash, sender, receiver, amount, token_denom, height))
        #To push message type - market order    
        elif msg_typ == "/comdex.liquidity.v1beta1.MsgMarketOrder":
            orderer = txs_res_arr[i]["tx"]["body"]["messages"][0]["orderer"]
            offer_coin_denom = txs_res_arr[i]["tx"]["body"]["messages"][0]["offer_coin"]["denom"]
            offer_coin_amount = txs_res_arr[i]["tx"]["body"]["messages"][0]["offer_coin"]['amount']
            demand_coin_denom = txs_res_arr[i]["tx"]["body"]["messages"][0]["demand_coin_denom"]
            demand_coin_amount = txs_res_arr[i]["tx"]["body"]["messages"][0]["amount"]
            
            cur.execute("INSERT INTO market_order (txs_hash, orderer, offer_coin_denom, offer_coin_amount, demand_coin_denom, demand_coin_amount, height) VALUES (%s, %s, %s, %s, %s, %s, %s)",
                        (txs_hash, orderer, offer_coin_denom, offer_coin_amount, demand_coin_denom, demand_coin_amount, height))
        #To push message type-limit order    
        elif msg_typ == "/comdex.liquidity.v1beta1.MsgLimitOrder":
            orderer = txs_res_arr[i]["tx"]["body"]["messages"][0]["orderer"]
            offer_coin_denom = txs_res_arr[i]["tx"]["body"]["messages"][0]["offer_coin"]["denom"]
            offer_coin_amount = txs_res_arr[i]["tx"]["body"]["messages"][0]["offer_coin"]['amount']
            demand_coin_denom = txs_res_arr[i]["tx"]["body"]["messages"][0]["demand_coin_denom"]
            price = txs_res_arr[i]["tx"]["body"]["messages"][0]["price"]
            amount = txs_res_arr[i]["tx"]["body"]["messages"][0]["amount"]
            

            cur.execute("INSERT INTO limit_order (txs_hash, orderer, offer_coin_denom, demand_coin_denom, price, amount) VALUES (%s,%s,%s,%s,%s,%s)",
                        (txs_hash, orderer, offer_coin_denom, demand_coin_denom, price, amount))
            
        Dir.commit()  

#To change the block height
while True:
    try:
        block_height = requests.get("https://rpc.comdex.one/status").json()["result"]["sync_info"]["latest_block_height"]
        get_txs_request(block_height)
    except Exception as e:
        print("Error occurred: ", e)
        pass
    time.sleep(6)



2
printing_> 0
printing_> 1
2
printing_> 0
printing_> 1
1
printing_> 0
1
printing_> 0
1
printing_> 0
0
2
printing_> 0
printing_> 1
1
printing_> 0
0
0
1
printing_> 0
0
3
printing_> 0
printing_> 1
printing_> 2
2
printing_> 0
printing_> 1
0
1
printing_> 0
0
0
0
0
0
2
printing_> 0
printing_> 1
0
0
1
printing_> 0
1
printing_> 0
0
0
0
0
1
printing_> 0
2
printing_> 0
Error occurred:  column "height" of relation "transfer" does not exist
LINE 1: ...(txs_hash, sender, receiver, amount, token_denom, height) VA...
                                                             ^

0
2
printing_> 0
Error occurred:  current transaction is aborted, commands ignored until end of transaction block

1
printing_> 0
Error occurred:  current transaction is aborted, commands ignored until end of transaction block

0
3
printing_> 0
Error occurred:  current transaction is aborted, commands ignored until end of transaction block

1
printing_> 0
Error occurred:  current transaction is aborted, commands ignored unt